In [3]:
import keras
from keras import models, layers

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtyp

In [4]:
from keras import backend
from keras.datasets import mnist

In [6]:
# 데이터를 맞추는 과정

In [7]:
# 데이터 획득
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [8]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [9]:
# 이미지 크기
img_rows, img_cols = x_train.shape[1:]
img_rows, img_cols

(28, 28)

In [10]:
# 채널의 위치 조정
backend.image_data_format()

'channels_last'

In [11]:
if backend.image_data_format() == 'channel_first':
    
    x_train = x_train.reshape( x_train.shape[0], 1, img_rows, img_cols )
    x_test  = x_test.reshape(  x_test.shape[0], 1, img_rows, img_cols )
    input_shape = ( 1, img_rows, img_cols )
    
else:
    # 흑백 이미지의 채널 정보를 처리하기 위해서는 추가 차원을 넣어야 한다
    # 컬러 이미지는 RGB색상을 다루는 채널 정보가 이미지 데이터에 포함되어 있다
    # 흑백은 채널 정보가 없어서, 입력 데이터 차원을 하나 더 추가해야 한다
    # 채널의 위치를 확인해서 앞쪽이면 앞으로, 뒤쪽이면 뒤로 배치
    x_train = x_train.reshape( x_train.shape[0], img_rows, img_cols, 1 )
    x_test = x_test.reshape( x_test.shape[0], img_rows, img_cols, 1 )
    input_shape = ( img_rows, img_cols, 1 )
    
    

In [12]:
x_train =  x_train.astype('float32')
x_test =  x_test.astype('float32')

In [13]:
x_train /= 255
x_test /= 255

In [14]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [15]:
a = set(y_train)
label_nums = len(a)

In [43]:
# 벡터화
# 레이블의 분류형 변수로 특성 이진화 처리
y_train = keras.utils.to_categorical( y_train, label_nums )
y_test = keras.utils.to_categorical( y_test, label_nums )

In [44]:
y_train[:2]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [45]:
x_train.shape, y_train.shape

((60000, 28, 28, 1), (60000, 10))

In [46]:
# 여기까지가 데이터를 준비하는 과정

In [47]:
# 모델 생성
model = models.Sequential()

In [48]:
# 학습공층 생성
# 32 : 출력 채널
# 커널, 필터, 가중치 -> 3x3, 활성화함수 지정, 편향은 생략
model.add( layers.Conv2D( 32, kernel_size=(3,3), activation='relu', input_shape=input_shape ) ) # 텐서플로우를 이용한 CNN 구현에서 합성곱층 만드는 것이랑 같은 것이다


In [49]:
# 풀링층
model.add( layers.MaxPool2D( pool_size=(2,2) ) )

In [50]:
# 과적합 방지( 개인이 조합해서 만드는 부분이다. )
# 0.25 : 설정값 (하이퍼파라미터)
model.add( layers.Dropout(0.25) )

In [51]:
# 합성곱2
model.add( layers.Conv2D( 64, (3,3), activation='relu' ) ) # kenel_size 를 빼도 무방한 이유는 2번째 자리가 원래 kenel_size자리이기 떄문에 굳이 풀 네임 안써도 된다

In [52]:
# 풀링층2
model.add( layers.MaxPool2D(pool_size=(2,2)) )

In [53]:
# 과적합 방지( 개인이 조합해서 만드는 부분이다. )
# 0.25 : 설정값 (하이퍼파라미터)
model.add( layers.Dropout(0.25) )

In [54]:
# 데이터를 편다
model.add(layers.Flatten())

In [55]:
# 전결합층
# 128 : 출력 채널 설정
model.add(layers.Dense(128, activation='relu'))

In [56]:
# 과적합 방지( 개인이 조합해서 만드는 부분이다. )
# 0.5 : 설정값 (하이퍼파라미터)
model.add( layers.Dropout(0.5) )

In [57]:
# 출력층
model.add( layers.Dense( label_nums, activation='softmax' ) )

In [58]:
# 손실함수 : 크로스엔트로피
# 최적화 : 평균제곱근오차 : rmsprop
# 정확도 : 
model.compile( loss=keras.losses.categorical_crossentropy,
               optimizer='rmsprop', metrics=['accuracy'])

In [59]:
# 학습 사이즈
batch_size = 128
# 수행횟수
epochs = 10

In [60]:
# 훈련데이터 중에 검증용 20% 사용
his = model.fit( x_train, y_train, batch_size=batch_size, epochs=epochs, 
                 validation_split=0.2 )

Instructions for updating:
Use tf.cast instead.
Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 70s 1ms/step - loss: 0.3756 - accuracy: 0.8805 - val_loss: 0.0889 - val_accuracy: 0.9742
Epoch 2/10
48000/48000 [==============================] - 66s 1ms/step - loss: 0.1289 - accuracy: 0.9613 - val_loss: 0.0581 - val_accuracy: 0.9825
Epoch 3/10
48000/48000 [==============================] - 67s 1ms/step - loss: 0.0954 - accuracy: 0.9709 - val_loss: 0.0508 - val_accuracy: 0.9849
Epoch 4/10
48000/48000 [==============================] - 68s 1ms/step - loss: 0.0783 - accuracy: 0.9766 - val_loss: 0.0426 - val_accuracy: 0.9883
Epoch 5/10
48000/48000 [==============================] - 67s 1ms/step - loss: 0.0692 - accuracy: 0.9790 - val_loss: 0.0425 - val_accuracy: 0.9882
Epoch 6/10
48000/48000 [==============================] - 66s 1ms/step - loss: 0.0628 - accuracy: 0.9818 - val_loss: 0.0410 - val_accuracy: 0.9883
Epoch 7/10
48000/480

In [61]:
score = model.evaluate( x_test, y_test )
score

10000/10000 [==============================] - 4s 446us/step


[0.025363308865006547, 0.9911999702453613]